In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q={}+economic+times&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [3]:
monitored_tickers = ['reliance', 'tatamotors', 'hindalco', 'TCS']

In [4]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'reliance': ['/?sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQOwgC',
  '/search?q=reliance+economic+times&tbm=nws&ie=UTF-8&gbv=1&sei=OAmLZJLcG_el5OUP9reLKA',
  '/search?q=reliance+economic+times&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUIBSgA',
  '/search?q=reliance+economic+times&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUIBygC',
  '/search?q=reliance+economic+times&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICCgD',
  'https://maps.google.com/maps?q=reliance+economic+times&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICSgE',
  '/search?q=reliance+economic+times&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICigF',
  '/search?q=reliance+economic+times&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICygG',
  '/advanced_search',
  '/search?q=reliance+economic+times&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkG

In [5]:
raw_urls.keys()

dict_keys(['reliance', 'tatamotors', 'hindalco', 'TCS'])

In [6]:
raw_urls.values()

dict_values([['/?sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQOwgC', '/search?q=reliance+economic+times&tbm=nws&ie=UTF-8&gbv=1&sei=OAmLZJLcG_el5OUP9reLKA', '/search?q=reliance+economic+times&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUIBSgA', '/search?q=reliance+economic+times&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUIBygC', '/search?q=reliance+economic+times&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICCgD', 'https://maps.google.com/maps?q=reliance+economic+times&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICSgE', '/search?q=reliance+economic+times&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICigF', '/search?q=reliance+economic+times&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQ_AUICygG', '/advanced_search', '/search?q=reliance+economic+times&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwiS2MarqMX_AhX3ErkGHfbbAgUQpwUIDQ', '

In [7]:
import re

In [8]:
exclude_list = ['maps' ,'policies' ,'preferences', 'accounts', 'support']

In [9]:
def strip_unwanted_urls(urls,exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)',url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [10]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'reliance': ['https://economictimes.indiatimes.com/industry/energy/oil-gas/reliance-suspends-sikka-diesel-exports-due-to-cyclone-traders/articleshow/100944683.cms',
  'https://economictimes.indiatimes.com/news/company/corporate-trends/reliance-jumps-8-spots-to-45th-rank-on-forbes-global-2000-list/articleshow/100959961.cms',
  'https://economictimes.indiatimes.com/markets/stocks/recos/buy-reliance-industries-target-price-rs-2579-icici-direct/articleshow/101008660.cms',
  'https://telecom.economictimes.indiatimes.com/news/industry/jios-pramod-k-mittal-to-continue-as-coai-chairperson-in-fy24/100996281',
  'https://economictimes.indiatimes.com/markets/expert-view/heres-why-rahul-sharma-is-bullish-on-rpg-life/articleshow/101014376.cms',
  'https://economictimes.indiatimes.com/tech/technology/reliances-wholesale-format-fires-1000-a-bigger-layoff-round-likely/articleshow/100428603.cms',
  'https://economictimes.indiatimes.com/markets/stocks/news/reliance-in-talks-for-2-billion-loan-to-fuel-e

In [11]:
def scrape_and_process(URLs):
    articles = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:300]
        article = ' '.join(words)
        articles.append(article)
    return articles

In [12]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'reliance': ['Don’t miss out on ET Prime stories! Get your daily dose of business updates on WhatsApp. click here! India’s retail inflation slowed more than expected to a 25-month low of 4.25% in May, close to the RBI’s target 4% rate. But experts don’t expect the central bank to start cutting interest rates any time soon. India’s job market is likely to see an uptick in July-September, signalling a positive turnaround after two consecutive quarters of cooling down, a survey of 3,020 employers has revealed. The government has asked Chinese mobile phone manufacturers to induct Indian equity partners in their local operations, said three executives who attended meetings at which the matter was conveyed. Download The Economic Times News App to get Daily Market Updates & Live Business News. Trending Now Popular Categories Hot on Web In Case you missed it Top Calculators Top Searched Companies Most Searched IFSC Codes Top Prime Articles Top Slideshow Top Videos Top Trending Topics Top Defi

In [13]:
! pip install transformers

In [14]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from tensorflow.python.keras.saving.hdf5_format import save_attributes_to_hdf5_group
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\msi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\msi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [15]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article,return_tensors='pt')
        output = model.generate(input_ids, max_length=130, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [16]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'reliance': ['India’s retail inflation slowed more than expected to a 25-month low of 4.25% in May. Experts don’t expect the central bank to start cutting interest rates any time soon. The government has asked Chinese mobile phone manufacturers to induct Indian equity partners in their local operations.',
  'Four state-owned financial entities are understood to be working on inviting formal bids for selling their stakes in UTI Asset Management Company. Hundreds of former employees from some of India’s biggest unicorns who have left to start up on their own are creating a ripple wave of internet enterprise. Akasa Air is looking to raise $75-100 million by offering fresh shares.',
  'Akasa Air, backed by late billionaire Rakesh Jhunjhunwala, is looking to raise $75-100 million by offering fresh shares. Hundreds of former employees from some of India’s biggest unicorns who have left to start up on their own.',
  'Delhi-based telecom industry association concluded its Annual General Body 

In [17]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'reliance': [{'label': 'NEGATIVE', 'score': 0.9959989786148071},
  {'label': 'POSITIVE', 'score': 0.9967479705810547},
  {'label': 'POSITIVE', 'score': 0.9538601040840149},
  {'label': 'POSITIVE', 'score': 0.9969134330749512},
  {'label': 'POSITIVE', 'score': 0.9980120658874512},
  {'label': 'NEGATIVE', 'score': 0.9959989786148071},
  {'label': 'POSITIVE', 'score': 0.9984074234962463},
  {'label': 'POSITIVE', 'score': 0.9980120658874512},
  {'label': 'NEGATIVE', 'score': 0.9733965396881104},
  {'label': 'NEGATIVE', 'score': 0.9959989786148071},
  {'label': 'NEGATIVE', 'score': 0.9933997392654419}],
 'tatamotors': [{'label': 'POSITIVE', 'score': 0.9984456896781921},
  {'label': 'NEGATIVE', 'score': 0.9933997392654419},
  {'label': 'NEGATIVE', 'score': 0.9933997392654419},
  {'label': 'NEGATIVE', 'score': 0.9535959362983704},
  {'label': 'NEGATIVE', 'score': 0.9399324059486389},
  {'label': 'NEGATIVE', 'score': 0.9933997392654419},
  {'label': 'NEGATIVE', 'score': 0.9933997392654419},
 

In [19]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output


In [20]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['reliance',
  'India’s retail inflation slowed more than expected to a 25-month low of 4.25% in May. Experts don’t expect the central bank to start cutting interest rates any time soon. The government has asked Chinese mobile phone manufacturers to induct Indian equity partners in their local operations.',
  'NEGATIVE',
  0.9959989786148071,
  'https://economictimes.indiatimes.com/industry/energy/oil-gas/reliance-suspends-sikka-diesel-exports-due-to-cyclone-traders/articleshow/100944683.cms'],
 ['reliance',
  'Four state-owned financial entities are understood to be working on inviting formal bids for selling their stakes in UTI Asset Management Company. Hundreds of former employees from some of India’s biggest unicorns who have left to start up on their own are creating a ripple wave of internet enterprise. Akasa Air is looking to raise $75-100 million by offering fresh shares.',
  'POSITIVE',
  0.9967479705810547,
  'https://economictimes.indiatimes.com/news/company/corporate-trend

In [21]:
final_output.insert(0,['Ticker','Summary', 'Label', 'Confidence', 'URL'])

In [22]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['reliance',
  'India’s retail inflation slowed more than expected to a 25-month low of 4.25% in May. Experts don’t expect the central bank to start cutting interest rates any time soon. The government has asked Chinese mobile phone manufacturers to induct Indian equity partners in their local operations.',
  'NEGATIVE',
  0.9959989786148071,
  'https://economictimes.indiatimes.com/industry/energy/oil-gas/reliance-suspends-sikka-diesel-exports-due-to-cyclone-traders/articleshow/100944683.cms'],
 ['reliance',
  'Four state-owned financial entities are understood to be working on inviting formal bids for selling their stakes in UTI Asset Management Company. Hundreds of former employees from some of India’s biggest unicorns who have left to start up on their own are creating a ripple wave of internet enterprise. Akasa Air is looking to raise $75-100 million by offering fresh shares.',
  'POSITIVE',
  0.9967479705810547,
  'https://eco

In [26]:
import csv
with open('stocksummaries.csv', mode='w', newline='', encoding="utf-8") as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)